In [10]:
import torch.nn as nn
import argparse
from util import load_UT_HAR_data, load_ReWiS_data, load_ReWiS_data_split, load_ReWiS_data_fewshot, load_Home_data_fewshot
from util import load_UT_HAR_supervised_model, load_ReWiS_supervised_model
import few_shot
import torch
import numpy as np
import supervised
import torch.backends.cudnn as cudnn
import random
import proto
from config import param
import os
import pandas as pd
from datetime import datetime
import wandb 
import main

''' 
fix seed
'''
torch.manual_seed(0)
torch.cuda.manual_seed(0)   
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

In [11]:
wandb.login()

True

In [12]:
os.environ["WANDB_AGENT_DISABLE_FLAPPING"] = "true"
os.environ["WANDB_AGENT_MAX_INITIAL_FAILURES"] = "20"

In [13]:
def main() : 
        now = str(datetime.now())
        root = './Data'

        learning = "few-shot"
        dataset = "ReWiS"
        model = "ProtoNet"
        max_epoch = 7
        epoch_size = 500
        train_way = 4
        test_way = 4


        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        train_x, train_y, test_x, test_y = load_ReWiS_data_fewshot(root)
        model = proto.load_protonet_conv(
                        x_dim=(1, 242, 242),
                        hid_dim=64,
                        z_dim=64,
                        )
        wandb.init (
                        project="Meta-Transformer",
                        # group=self.config['fs_model'],
                        # group="Test Sweep",
                        group="Test Sweep ReWiS ProtoNet",
                        name=now,
                        notes= "train : few_shot_datasets/ReWis/m1c4_PCA_test_80/train_A\n" +
                        "test : few_shot_datasets/ReWis/m1c4_PCA_test_80/test_A3",

                        # track hyperparameters and run metadata
                        config = {
                        "architecture": "ProtoNet",
                        "max_epoch": max_epoch,
                        "epoch_size" : epoch_size
                        }
                )
        w_config = wandb.config

        print("train_way: " + str(train_way))
        print("train_support : " + str(w_config.train_support))
        print("train_query : " + str(w_config.train_query))

        train_accuracy_history, train_loss_history = few_shot.train(
                model = model, 
                learning_rate=1e-3, 
                train_x = train_x, 
                train_y = train_y,
                n_way = train_way,
                n_support = w_config.train_support, 
                n_query = w_config.train_query,
                max_epoch = max_epoch,
                epoch_size = epoch_size,
                device = device,
                )   

        print("test_way: " + str(test_way))
        print("test_support : " + str(w_config.test_support))
        print("test_query : " + str(w_config.test_query))

        conf_mat, test_acc = few_shot.test(
                model = model,
                test_x = test_x,
                test_y = test_y,
                n_way = test_way,
                n_support = w_config.test_support,
                n_query = w_config.test_query,
                test_episode = 1,
                device = device,
                )

                # Result/learning/dataset/model_name/max_epoch/epoch_size/train.csv
                # Result/learning/dataset/model_name/max_epoch/epoch_size/test.csv
                # Result/learning/dataset/model_name/max_epoch/epoch_size/conf.csv
                # model_out = f'Result/{args.learning}/{args.dataset}/{MHz}_{train_dir}_{test_dir}/{args.model}/train_s{param['train_support']}_train_q{param['train_query']}_test_s{param['test_support']}_test_q{param['test_query']}/{param['max_epoch']}_{param['epoch_size']}_{test_acc:.3f}/'
        model_out = (
                        f"Result/{learning}/{dataset}/{model}/"
                        f"train_s{w_config.train_support}_train_q{w_config.train_query}_"
                        f"test_s{w_config.test_support}_test_q{w_config.test_query}/"
                        f"{max_epoch}_{epoch_size}_{test_acc:.3f}/"
                        )
        print(model_out)
                # model_out = 'Result/{}/{}/{}/{}/{}/train_s{}_train_q{}_test_s{}_test_q{}/{}/'.format(args.learning, args.dataset, args.model, param['max_epoch'], param['epoch_size'],param['train_support'],param['train_query'],param['test_support'],param['test_query'],test_acc)
        if not os.path.exists(model_out):
                os.makedirs(model_out)
                
        train_history = pd.DataFrame({'Epoch': range(0, max_epoch),
                        'Accuracy': train_accuracy_history,
                        'Loss': train_loss_history})
        test_history = pd.DataFrame({'Test Accuracy': [test_acc]})
        confusion_matrix = pd.DataFrame(conf_mat.numpy())
                
        train_history.to_csv(model_out+ 'train.csv', index=False)
        test_history.to_csv(model_out+ 'test.csv',index=False)
        confusion_matrix.to_csv(model_out+'confusion.csv', index=True)    
        torch.save(model.state_dict(),model_out + 'model.pt')
        wandb.finish()
        

In [14]:
sweep_configuration = {
    'project': 'Meta-Transformer',
    'name': 'test sweep grid',
    'entity' : 'pjs990301',
    'method' : 'bayes',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'test_acc'
    },
    'parameters': {
        'train_support': {'values': [3, 4, 5, 10, 15, 20, 25, 30]},
        'train_query': {'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
        'test_support': {'values': [3, 4, 5, 10, 15, 20, 25, 30]},
        'test_query': {'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]},
     }
}

In [15]:
sweep_id = wandb.sweep(
    sweep=sweep_configuration, 
    project='Meta-Transformer',
    entity='pjs990301'
    )

Create sweep with ID: 0crsyhdq
Sweep URL: https://wandb.ai/pjs990301/Meta-Transformer/sweeps/0crsyhdq


In [17]:
wandb.agent(sweep_id, function=main)

wandb: Agent Starting Run: 9onmosc1 with config:
wandb: 	test_query: 1
wandb: 	test_support: 3
wandb: 	train_query: 1
wandb: 	train_support: 4


using dataset: ReWiS DATA Few shot


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


train_way: 4
train_support : 4
train_query : 1


Epoch 1 train: 100%|██████████| 500/500 [00:23<00:00, 21.04it/s]


...

Epoch 7 -- Acc: 0.25000 Loss: 0.125445613
test_way: 4
test_support : 3
test_query : 1


test: 100%|██████████| 1/1 [00:00<00:00, 5184.55it/s]

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


acc,▁▁▁▁▁▁▁
loss,█▁▁▁▁▁▁
acc,0.25
loss,0.12545


Run 9onmosc1 errored: ZeroDivisionError('float division by zero')
wandb: ERROR Run 9onmosc1 errored: ZeroDivisionError('float division by zero')
